In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import evals2
import random

MAX_LENGTH = 10

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

import random
import math
import time

ModuleNotFoundError: No module named 'torchtext'

In [ ]:
def parse_command(commands, u):
    primitive = []
    current_words = []
    for i, word in enumerate(commands):
        # print(commands[:i+1], current_words, primitive)
        if word == 'walk':
            current_words.append(word)
        elif word == 'look':
            current_words.append(word)
        elif word == 'run':
            current_words.append(word)
        elif word == 'jump':
            current_words.append(word)

        elif current_words == ['turn'] and word == 'left':
            primitive.append('turn left')
            current_words = []
        elif current_words == ['turn'] and word == 'right':
            primitive.append('turn right')
            current_words = []

        elif len(current_words) > 1 and current_words[0] in u and current_words[1] == 'opposite' and word == 'left':
            primitive.append([['turn left', 'turn left'], current_words[0]])
            current_words = []
        elif len(current_words) > 1 and current_words[0] in u and current_words[1] == 'opposite' and word == 'right':
            primitive.append([['turn right', 'turn right'], current_words[1]])
            current_words = []

        elif len(current_words) > 1 and current_words[0] == "turn" and current_words[1] == 'opposite' and word == 'left':
            primitive.append([['turn left', 'turn left'], current_words[0]])
            current_words = []
        elif len(current_words) > 1 and current_words[0] == "turn" and current_words[1] == 'opposite' and word == 'right':
            primitive.append([['turn left', 'turn left'], current_words[1]])
            current_words = []

        elif current_words == ['turn', 'around'] and word == 'left':
            primitive.append(['turn left', 'turn left', 'turn left', 'turn left'])
            current_words = []
        elif current_words == ['turn', 'around'] and word == 'right':
            primitive.append(['turn right', 'turn right', 'turn right', 'turn right'])
            current_words = []

        elif len(current_words) > 1 and current_words[0] in u and current_words[1] == 'around' and word == 'left':
            primitive.append(['turn left', current_words[0], 'turn left', current_words[0], 
                'turn left', current_words[0], 'turn left', current_words[0]])
            current_words = []
        elif len(current_words) > 1 and current_words[0] in u and current_words[1] == 'around' and word == 'right':
            primitive.append(['turn right', current_words[0], 'turn right', current_words[0], 
                'turn right', current_words[0], 'turn right', current_words[0]])
            current_words = []

        elif current_words != [] and current_words[0] in u and word == 'left':
            primitive.append([current_words[0], 'left'])
            current_words = []
        elif current_words != [] and current_words[0] in u and word == 'right':
            primitive.append([current_words[0], 'right'])
            current_words = []

        elif word == 'turn':
            current_words.append("turn")
        elif word == 'around':
            current_words.append("around")
        elif word == 'opposite':
            current_words.append("opposite")       
        elif word == 'twice':
            if len(current_words) == 1 and current_words[0] in u:
                primitive.append(current_words[0])
            current_words = []
            primitive = primitive * 2
        elif word == 'thrice':
            if len(current_words) == 1 and current_words[0] in u:
                primitive.append(current_words[0])
            current_words = []
            primitive = primitive * 3
        elif word == 'and':
            if current_words != []:
                primitive += current_words
            return [primitive, parse_command(commands[i + 1:], u)] 
        elif word == 'after':
            if current_words != []:
                primitive += current_words
            return [parse_command(commands[i + 1:], u), primitive]
    if current_words != []:
        primitive += current_words
    return primitive

def parse_line(line):

    command, actions = line.split("IN: ")[1].split(" OUT: ")
    u = ['walk', 'look', 'run', 'jump']
    return command, actions, parse_command(command.split(), u)

In [ ]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [ ]:
file = open('baroni_dataset.txt', 'r')
pairs = []
ground_truth_examples = []
for line in file.readlines():
    command, actions, comp = parse_line(line)
    ground_truth_examples.append(comp)
    pairs.append((command, actions))

input_lang = Lang("in")
output_lang = Lang("out")
file.close()

In [ ]:
print("Read %s sentence pairs" % len(pairs))
print("Counting words...")
for pair in pairs:
    input_lang.addSentence(pair[0])
    output_lang.addSentence(pair[1])
print("Counted words:")
print(input_lang.name, input_lang.n_words)
print(output_lang.name, output_lang.n_words)

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden, 0

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def trainIters(encoder, decoder, training_pairs, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

n_iters = 75000
training_pairs = [tensorsFromPair(random.choice(pairs))
                  for i in range(n_iters)]
trainIters(encoder1, decoder1, 75000, print_every=5000)

In [ ]:
n_iter = 19000
training_pairs = [tensorsFromPair(pairs[i])
                  for i in range(n_iter)]
testing_pairs = [tensorsFromPair(pairs[i])
                  for i in range(n_iter, len(pairs))]
testing_pairs_display = [pairs[i]
                  for i in range(n_iter, len(pairs))]

In [ ]:
hidden_size = 256
encoder2 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder2 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, decoder1, training_pairs, n_iter, print_every=4000)

In [ ]:
hidden_size = 256
encoder3 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder3 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, training_pairs, n_iter, print_every=4000)

In [ ]:
def evaluate(encoder, input_tensor, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        return encoder_hidden
        

class Composition(nn.Module):
    def forward(self, x):
        return np.sum(np.array([x]))

    
def evaluate_comp(encoder, testing_pairs):
    representations = []
    expressions = []
#     for i in range(len(testing_pairs)):
    for i in range(1000):
#         print(ground_truth_examples[n_iter + i], testing_pairs_display[0])
        expressions.append(ground_truth_examples[n_iter + i])
        representations.append(evaluate(encoder, testing_pairs[i][0]))
    return evaluatec(representations, expressions, Composition(), evals2.CosDist(), steps=500, zero_init=False)

In [ ]:
def flatten(l):
    if not isinstance(l, tuple) and not isinstance(l, list):
        return (l,)

    out = ()
    for ll in l:
        out = out + flatten(ll)
    return out

class Objective(nn.Module):
    def __init__(self, vocab, repr_size, comp_fn, err_fn, zero_init):
        super().__init__()
        self.emb = nn.Embedding(len(vocab), repr_size)
        if zero_init:
            self.emb.weight.data.zero_()
        self.comp = comp_fn
        self.err = err_fn

    def compose(self, e):
        if isinstance(e, tuple):
            args = (self.compose(ee) for ee in e)
            return self.comp(*args)
        if isinstance(e, list):
            args = [self.compose(ee) for ee in e]
            return self.comp(args)
        return self.emb(e)

    def forward(self, rep, expr):
        return self.err(self.compose(expr), rep)

def evaluatec(reps, exprs, comp_fn, err_fn, quiet=False, steps=400, include_pred=False, zero_init=True):
    vocab = {}
    for expr in exprs:
        toks = flatten(expr)
        for tok in toks:
            if tok not in vocab:
                vocab[tok] = len(vocab)

    def index(e):
        if isinstance(e, tuple):
            return tuple(index(ee) for ee in e)
        if isinstance(e, list):
            return [index(ee) for ee in e]
        return torch.LongTensor([vocab[e]])
    treps = [torch.FloatTensor(r.data.cpu().numpy()[0]) for r in reps]
    texprs = [index(e) for e in exprs]
    
    obj = Objective(vocab, reps[0].data.cpu().numpy()[0][0].size, comp_fn, err_fn, zero_init)
    opt = optim.RMSprop(obj.parameters(), lr=0.01)

    for t in range(steps):
        opt.zero_grad()
        errs = [obj(r, e) for r, e in zip(treps, texprs)]
        loss = sum(errs)
        loss.backward(retain_graph=True)
        if not quiet and t % 100 == 0:
            print(loss.item())
        opt.step()

    final_errs = [err.item() for err in errs]
    if include_pred:
        lexicon = {
            k: obj.emb(torch.LongTensor([v])).data.cpu().numpy()
            for k, v in vocab.items()
        }
        composed = [obj.compose(t) for t in texprs]
        return final_errs, lexicon, composed, sum(errs)
    else:
        return final_errs, sum(errs)

In [ ]:
evaluate_comp(encoder2, testing_pairs)

In [ ]:
evaluate_comp(encoder3, testing_pairs)